In [3]:
# --- SETUP AWAL ---
# Python ≥3.5 diperlukan
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 diperlukan
import sklearn
assert sklearn.__version__ >= "0.20"

# Impor library umum
import numpy as np
import pandas as pd
import os

# Untuk membuat plot yang konsisten
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Direktori untuk menyimpan gambar
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Menyimpan gambar", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

"""
# Bab 7: Ensemble Learning dan Random Forests

## Penjelasan Teoretis: Konsep Dasar Ensemble Learning
Ensemble Learning adalah teknik di mana kita menggabungkan prediksi dari sekelompok *predictor* (disebut *ensemble*) untuk mendapatkan prediksi yang lebih baik daripada prediksi dari masing-masing predictor tunggal. Fenomena ini mirip dengan "kebijaksanaan orang banyak" (*wisdom of the crowd*).

Contohnya, jika kita melatih sekelompok classifier Decision Tree pada subset data yang berbeda-beda, kemudian kita ambil suara mayoritas dari prediksi mereka, kita akan mendapatkan model yang disebut **Random Forest**.

Metode ensemble umumnya sangat efektif dan sering menjadi bagian dari solusi pemenang dalam kompetisi Machine Learning.
"""

# --- 1. Voting Classifiers ---
"""
Cara paling sederhana untuk membuat ensemble adalah dengan menggabungkan prediksi dari beberapa classifier yang berbeda dan memilih kelas yang paling banyak mendapatkan suara (*majority vote*). Ini disebut **Hard Voting Classifier**.

Hebatnya, akurasi dari *voting classifier* ini seringkali lebih tinggi daripada classifier terbaik dalam ensemble tersebut. Hal ini bisa terjadi jika para classifier cukup beragam dan membuat jenis kesalahan yang berbeda-beda.

Jika semua classifier dapat mengestimasi probabilitas, kita bisa menggunakan **Soft Voting Classifier**. Di sini, kita merata-ratakan probabilitas kelas dari semua classifier dan memilih kelas dengan probabilitas rata-rata tertinggi. Soft voting seringkali berkinerja lebih baik karena memberikan bobot lebih pada prediksi yang sangat "yakin".
"""
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

# Hard Voting
voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf_hard.fit(X_train, y_train)

print("--- Akurasi Voting Classifier ---")
for clf in (log_clf, rnd_clf, svm_clf, voting_clf_hard):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"Akurasi {clf.__class__.__name__}: {accuracy_score(y_test, y_pred):.4f}")

# Soft Voting
# SVM perlu diatur probability=True untuk bisa melakukan soft voting
svm_clf_soft = SVC(gamma="scale", probability=True, random_state=42)
voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf_soft)],
    voting='soft'
)
voting_clf_soft.fit(X_train, y_train)
y_pred_soft = voting_clf_soft.predict(X_test)
print(f"Akurasi Soft Voting Classifier: {accuracy_score(y_test, y_pred_soft):.4f}")


# --- 2. Bagging and Pasting ---
"""
Pendekatan lain untuk membuat ensemble adalah menggunakan algoritma yang sama tetapi melatihnya pada subset acak yang berbeda dari set pelatihan.
- **Bagging** (*Bootstrap Aggregating*): Sampling dilakukan **dengan** penggantian.
- **Pasting**: Sampling dilakukan **tanpa** penggantian.

Setelah semua predictor dilatih, ensemble membuat prediksi dengan agregasi (suara terbanyak untuk klasifikasi, rata-rata untuk regresi). Metode ini mengurangi varians model.
"""
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Ensemble dari 500 Decision Tree
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42
)
bag_clf.fit(X_train, y_train)
y_pred_bag = bag_clf.predict(X_test)
print(f"\nAkurasi Bagging Classifier: {accuracy_score(y_test, y_pred_bag):.4f}")

"""
#### Out-of-Bag (oob) Evaluation
Dalam bagging, karena sampling dilakukan dengan penggantian, rata-rata hanya sekitar 63% instance yang terambil untuk setiap predictor. Sisa 37% yang tidak terambil disebut *out-of-bag (oob) instances*.

Kita bisa menggunakan instance oob ini sebagai validation set untuk mengevaluasi kinerja ensemble tanpa perlu memisahkan data validasi secara manual. Atur `oob_score=True`.
"""
bag_clf_oob = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40
)
bag_clf_oob.fit(X_train, y_train)
print(f"\nOOB Score: {bag_clf_oob.oob_score_:.4f}")


# --- 3. Random Forests ---
"""
**Random Forest** adalah ensemble dari Decision Trees, umumnya dilatih dengan metode bagging. Scikit-Learn menyediakan kelas `RandomForestClassifier` yang lebih nyaman dan teroptimisasi.

Random Forest menambahkan satu lapis keacakan lagi: saat memecah sebuah node, ia tidak mencari fitur terbaik dari *semua* fitur, melainkan dari *subset acak* fitur. Ini menghasilkan pohon yang lebih beragam dan (sekali lagi) menukar bias yang sedikit lebih tinggi dengan varians yang lebih rendah, yang secara umum menghasilkan model yang lebih baik.
"""
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)
print(f"\nAkurasi Random Forest: {accuracy_score(y_test, y_pred_rf):.4f}")


"""
#### Feature Importance
Salah satu keunggulan Random Forest adalah kemudahannya dalam mengukur pentingnya setiap fitur. Scikit-Learn menghitungnya dengan melihat seberapa besar sebuah fitur mengurangi *impurity* (ketidakmurnian) secara rata-rata di semua pohon dalam forest.
"""
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf_iris = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf_iris.fit(iris["data"], iris["target"])

print("\n--- Feature Importance pada Iris Dataset ---")
for name, score in zip(iris["feature_names"], rnd_clf_iris.feature_importances_):
    print(f"{name}: {score:.4f}")


# --- 4. Boosting ---
"""
**Boosting** adalah metode ensemble yang melatih predictor secara sekuensial, di mana setiap predictor baru mencoba memperbaiki kesalahan predictor sebelumnya.

#### a. AdaBoost (Adaptive Boosting)
Ide utamanya adalah membuat predictor baru lebih fokus pada instance yang salah diklasifikasikan oleh predictor sebelumnya. Ini dilakukan dengan meningkatkan bobot relatif dari instance yang salah diklasifikasikan.
"""
from sklearn.ensemble import AdaBoostClassifier

# DIBENERKEUN: Parameter `algorithm="SAMME.R"` geus teu dipaké dina versi Scikit-Learn anyar.
# 'SAMME' anu jadi default bakal otomatis ngagunakeun logika 'SAMME.R' upami base estimator
# ngadukung predict_proba(), saperti DecisionTreeClassifier.
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    learning_rate=0.5, random_state=42
)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print(f"\nAkurasi AdaBoost: {accuracy_score(y_test, y_pred_ada):.4f}")


"""
#### b. Gradient Boosting
Metode boosting populer lainnya. Alih-alih menyesuaikan bobot instance, Gradient Boosting melatih predictor baru pada **residual errors** (kesalahan sisa) yang dibuat oleh predictor sebelumnya.
"""
from sklearn.tree import DecisionTreeRegressor

# Membuat data untuk regresi
np.random.seed(42)
X_reg = np.random.rand(100, 1) - 0.5
y_reg = 3*X_reg[:, 0]**2 + 0.05 * np.random.randn(100)

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X_reg, y_reg)

y2 = y_reg - tree_reg1.predict(X_reg) # residual errors
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X_reg, y2)

y3 = y2 - tree_reg2.predict(X_reg) # residual errors dari model kedua
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X_reg, y3)

# Scikit-Learn menyediakan kelas yang lebih mudah
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

# Early Stopping untuk menemukan jumlah pohon optimal
errors = [mean_squared_error(y_test, y_pred)
          for y_pred in gbrt.staged_predict(X_test)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)
print(f"\nJumlah Pohon Optimal untuk GBRT: {bst_n_estimators}")

# --- 5. Stacking ---
"""
**Stacking** (*Stacked Generalization*) adalah metode ensemble yang didasarkan pada ide sederhana: daripada menggunakan fungsi agregasi trivial seperti *voting*, mengapa tidak melatih sebuah model untuk melakukan agregasi tersebut?

Model yang dilatih untuk menggabungkan prediksi ini disebut **blender** atau **meta learner**.
"""

# --- JAWABAN LATIHAN TEORETIS ---

"""
### Latihan: Jawaban Teoretis

**1. Jika Anda melatih lima model berbeda pada data yang sama persis, dan semuanya mencapai presisi 95%, adakah peluang untuk menggabungkannya untuk mendapatkan hasil yang lebih baik?**
**Ya, ada peluang**. Jika model-model tersebut cukup berbeda (misalnya, Logistic Regression, SVM, Random Forest) dan membuat jenis kesalahan yang berbeda, maka menggabungkannya (misalnya, dengan *voting classifier*) dapat menghasilkan kinerja yang lebih baik. Kesalahan dari satu model mungkin dapat dikoreksi oleh model lain.

**2. Apa perbedaan antara hard dan soft voting classifiers?**
- **Hard Voting**: Memprediksi kelas yang menerima suara mayoritas dari para classifier. Sederhana dan lugas.
- **Soft Voting**: Menghitung rata-rata probabilitas kelas dari semua classifier, lalu memprediksi kelas dengan probabilitas rata-rata tertinggi. Ini seringkali berkinerja lebih baik karena memberikan bobot lebih pada prediksi yang sangat "yakin".

**3. Apakah mungkin mempercepat pelatihan bagging ensemble dengan mendistribusikannya ke beberapa server? Bagaimana dengan pasting, boosting, Random Forests, atau stacking?**
- **Bagging/Pasting**: **Ya**. Setiap predictor dalam ensemble dilatih secara independen pada subset data yang berbeda, sehingga proses ini sangat mudah untuk diparalelkan.
- **Random Forests**: **Ya**. Ini pada dasarnya adalah bagging dari Decision Trees, jadi bisa diparalelkan.
- **Boosting**: **Tidak**. Metode boosting melatih predictor secara sekuensial, di mana setiap pohon bergantung pada hasil pohon sebelumnya. Ini membuatnya sulit untuk diparalelkan.
- **Stacking**: **Sebagian**. Predictor di layer yang sama bisa dilatih secara paralel. Namun, layer-layer harus dilatih secara sekuensial (layer 2 dilatih setelah layer 1 selesai).

**4. Apa manfaat dari evaluasi out-of-bag (oob)?**
Manfaatnya adalah kita bisa mendapatkan estimasi kinerja model pada data yang belum pernah dilihat tanpa perlu membuat *validation set* terpisah. Ini sangat efisien karena kita bisa menggunakan seluruh set pelatihan untuk pelatihan sekaligus mendapatkan evaluasi yang tidak bias.

**5. Apa yang membuat Extra-Trees lebih acak daripada Random Forests biasa? Bagaimana keacakan ekstra ini bisa membantu? Apakah Extra-Trees lebih lambat atau lebih cepat?**
- **Apa yang membuatnya lebih acak?**: Selain memilih subset fitur secara acak seperti Random Forest, Extra-Trees juga menggunakan *threshold* (nilai ambang batas) acak untuk memecah node, alih-alih mencari threshold terbaik.
- **Bagaimana ini membantu?**: Keacakan ekstra ini bertindak sebagai bentuk regularisasi, menukar bias yang sedikit lebih tinggi dengan varians yang jauh lebih rendah.
- **Lebih cepat atau lebih lambat?**: Extra-Trees **lebih cepat** untuk dilatih daripada Random Forests karena mencari threshold optimal untuk setiap fitur di setiap node adalah salah satu tugas yang paling memakan waktu.

**6. Jika ensemble AdaBoost Anda underfitting, hyperparameter mana yang harus Anda ubah dan bagaimana?**
Untuk mengatasi underfitting, Anda perlu meningkatkan kompleksitas model. Anda bisa:
- **Menaikkan `n_estimators`**: Menambah jumlah predictor dalam ensemble.
- **Mengurangi regularisasi pada *base estimator***: Misalnya, jika menggunakan Decision Tree, naikkan `max_depth`.
- **Menaikkan `learning_rate`**: Membuat setiap predictor memiliki kontribusi yang lebih besar.

**7. Jika ensemble Gradient Boosting Anda overfitting, haruskah Anda menaikkan atau menurunkan learning rate?**
Anda harus **menurunkan `learning_rate`**. `learning_rate` yang lebih rendah (disebut *shrinkage*) berarti setiap pohon memiliki kontribusi yang lebih kecil. Ini memaksa kita untuk menggunakan lebih banyak pohon (`n_estimators`) untuk menyesuaikan data, tetapi secara umum menghasilkan model yang generalisasinya lebih baik. Anda juga bisa menggunakan *early stopping* untuk menemukan jumlah pohon yang optimal dan mencegah overfitting.
"""

# --- LATIHAN PRAKTIS (Kerangka) ---
"""
### Latihan 8 & 9: Ensemble pada MNIST dan Stacking

*Catatan: Menjalankan kode ini bisa sangat lama, terutama pada CPU biasa.
Kode di bawah ini adalah kerangka untuk menyelesaikan latihan.
"""
print("\n--- Latihan 8 & 9: Ensemble dan Stacking pada MNIST ---")
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)

    X_mnist_train_full, X_mnist_test, y_mnist_train_full, y_mnist_test = mnist["data"][:60000], mnist["data"][60000:], mnist["target"][:60000], mnist["target"][60000:]
    y_mnist_train_full = y_mnist_train_full.astype(np.uint8)
    y_mnist_test = y_mnist_test.astype(np.uint8)
    
    # Buat validation set
    X_mnist_train, X_mnist_val, y_mnist_train, y_mnist_val = train_test_split(
        X_mnist_train_full, y_mnist_train_full, test_size=10000, random_state=42)

    # Latih beberapa classifier
    from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
    from sklearn.svm import LinearSVC

    random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
    extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
    svm_lin_clf = LinearSVC(max_iter=100, tol=20, random_state=42)

    estimators = [random_forest_clf, extra_trees_clf, svm_lin_clf]
    print("\nMelatih classifier individual...")
    for estimator in estimators:
        estimator.fit(X_mnist_train, y_mnist_train)
        y_pred = estimator.predict(X_mnist_val)
        print(f"Akurasi {estimator.__class__.__name__}: {accuracy_score(y_mnist_val, y_pred):.4f}")

    # Gabungkan dalam Voting Classifier
    named_estimators = [
        ("random_forest_clf", random_forest_clf),
        ("extra_trees_clf", extra_trees_clf),
        ("svm_clf", svm_lin_clf), # SVM tidak punya predict_proba, jadi kita pakai hard voting
    ]
    voting_clf = VotingClassifier(named_estimators, voting='hard')
    voting_clf.fit(X_mnist_train, y_mnist_train)
    y_pred_voting = voting_clf.predict(X_mnist_val)
    print(f"Akurasi Voting Classifier (validation): {accuracy_score(y_mnist_val, y_pred_voting):.4f}")

    # Evaluasi di test set
    y_pred_voting_test = voting_clf.predict(X_mnist_test)
    print(f"Akurasi Voting Classifier (test): {accuracy_score(y_mnist_test, y_pred_voting_test):.4f}")


    # Latihan 9: Stacking
    print("\nMembuat Stacking Ensemble...")
    # Buat dataset baru dari prediksi di validation set
    predictions = np.empty((len(X_mnist_val), len(estimators)), dtype=np.float32)
    for index, estimator in enumerate(estimators):
        predictions[:, index] = estimator.predict(X_mnist_val)

    # Latih blender
    blender_clf = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
    blender_clf.fit(predictions, y_mnist_val)
    print(f"OOB Score Blender: {blender_clf.oob_score_:.4f}")

    # Evaluasi stacking ensemble di test set
    X_test_predictions = np.empty((len(X_mnist_test), len(estimators)), dtype=np.float32)
    for index, estimator in enumerate(estimators):
        X_test_predictions[:, index] = estimator.predict(X_mnist_test)

    y_pred_stacking = blender_clf.predict(X_test_predictions)
    print(f"Akurasi Stacking Ensemble (test): {accuracy_score(y_mnist_test, y_pred_stacking):.4f}")


except Exception as e:
    print(f"\nGagal memuat atau melatih pada dataset MNIST. Mungkin ada masalah koneksi atau butuh waktu lama. Error: {e}")


--- Akurasi Voting Classifier ---
Akurasi LogisticRegression: 0.8640
Akurasi RandomForestClassifier: 0.8960
Akurasi SVC: 0.8960
Akurasi VotingClassifier: 0.9120
Akurasi Soft Voting Classifier: 0.9200

Akurasi Bagging Classifier: 0.9040

OOB Score: 0.8987

Akurasi Random Forest: 0.9120

--- Feature Importance pada Iris Dataset ---
sepal length (cm): 0.1125
sepal width (cm): 0.0231
petal length (cm): 0.4410
petal width (cm): 0.4234


C:\Users\raehu\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Akurasi AdaBoost: 0.8960

Jumlah Pohon Optimal untuk GBRT: 75

--- Latihan 8 & 9: Ensemble dan Stacking pada MNIST ---

Melatih classifier individual...
Akurasi RandomForestClassifier: 0.9683
Akurasi ExtraTreesClassifier: 0.9718
Akurasi LinearSVC: 0.0984
Akurasi Voting Classifier (validation): 0.9637
Akurasi Voting Classifier (test): 0.9629

Membuat Stacking Ensemble...
OOB Score Blender: 0.9697
Akurasi Stacking Ensemble (test): 0.9702
